In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time

def scrape_universities_to_csv(output_csv="universities.csv"):
    """
    Scrape the universities from univ.cc (world domain) in steps of 50,
    capturing university name, country, and storing the URL as a clickable
    Excel-friendly hyperlink in the CSV.
    """
    base_url = "https://univ.cc/search?dom=world&key=&start="

    # Open the CSV once in write mode and store the header
    with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        # Write header
        writer.writerow(["University", "Country", "URL"])

        # We go in steps of 50 results: 1, 51, 101, ... up to 7851
        for start in range(1, 7851, 50):
            url = f"{base_url}{start}"
            print(f"Scraping: {url}")
            response = requests.get(url)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, "html.parser")

            # Find the <ol> that contains the <li> items
            ol = soup.find("ol")
            if not ol:
                # If there's no <ol>, skip
                print(f"No <ol> found on page {url}")
                continue

            li_tags = ol.find_all("li")

            for li in li_tags:
                # Extract the anchor tag
                a_tag = li.find("a")
                if not a_tag:
                    continue

                university_name = a_tag.get_text(strip=True)
                university_href = a_tag.get("href", "").strip()

                # Country typically in parentheses at the end of the li text
                full_text = li.get_text(strip=True)
                # e.g.: "Some University (SomeCountry)"
                start_paren = full_text.rfind("(")
                end_paren = full_text.rfind(")")
                if start_paren != -1 and end_paren != -1 and end_paren > start_paren:
                    country = full_text[start_paren+1 : end_paren].strip()
                else:
                    country = "Unknown"

                # Create an Excel-friendly hyperlink formula for the URL
                # Format: =HYPERLINK("actual_link", "display_text")
                # You can choose what text to display. Here, we use the link itself.
                hyperlink_formula = f'=HYPERLINK("{university_href}", "{university_href}")'

                # Write the row
                writer.writerow([university_name, country, hyperlink_formula])

            # Optionally, sleep a little to avoid hammering the server
            # time.sleep(1)

    print(f"Scraping complete. All data saved to '{output_csv}'.")

if __name__ == "__main__":
    scrape_universities_to_csv("universities.csv")


Scraping: https://univ.cc/search?dom=world&key=&start=1
Scraping: https://univ.cc/search?dom=world&key=&start=51
Scraping: https://univ.cc/search?dom=world&key=&start=101
Scraping: https://univ.cc/search?dom=world&key=&start=151
Scraping: https://univ.cc/search?dom=world&key=&start=201
Scraping: https://univ.cc/search?dom=world&key=&start=251
Scraping: https://univ.cc/search?dom=world&key=&start=301
Scraping: https://univ.cc/search?dom=world&key=&start=351
Scraping: https://univ.cc/search?dom=world&key=&start=401
Scraping: https://univ.cc/search?dom=world&key=&start=451
Scraping: https://univ.cc/search?dom=world&key=&start=501
Scraping: https://univ.cc/search?dom=world&key=&start=551
Scraping: https://univ.cc/search?dom=world&key=&start=601
Scraping: https://univ.cc/search?dom=world&key=&start=651
Scraping: https://univ.cc/search?dom=world&key=&start=701
Scraping: https://univ.cc/search?dom=world&key=&start=751
Scraping: https://univ.cc/search?dom=world&key=&start=801
Scraping: https:/


## US Uni Scraper

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import time

def parse_states(html_text):
    """
    Given the HTML snippet that includes <option> tags for each US state,
    parse out a list of tuples: (domain, state_name, count).
    Example: ("edu_al", "Alabama", 39)
    """
    soup = BeautifulSoup(html_text, "html.parser")
    select_tag = soup.find("select", {"name": "dom"})
    if not select_tag:
        return []

    options = select_tag.find_all("option")

    # We'll store parsed results here
    states_data = []

    for opt in options:
        # Skip the "* Search in all U.S. States" or any that don't have 'value'
        if not opt.get("value") or opt["value"] == "world":
            continue

        domain = opt["value"].strip()  # e.g., "edu_al"
        text = opt.get_text(strip=True)  # e.g., "Alabama (39)"

        # We need to extract the state name and the count (in parentheses)
        # text is typically "Alabama (39)"
        match = re.match(r"^(.*?)\s*\((\d+)\)$", text)
        if match:
            state_name = match.group(1).strip()
            count = int(match.group(2))
            states_data.append((domain, state_name, count))
        else:
            # In case there's no match, skip or handle differently
            continue

    return states_data

def scrape_universities_for_state(domain, state_name, total_count, writer):
    """
    Given a state domain (e.g. 'edu_al'), state name ('Alabama'),
    and the total number of universities in that state,
    iterate over all pages (in steps of 50), scrape the data, and
    write CSV rows using the provided 'writer' (csv.writer).
    """
    base_search_url = "https://univ.cc/search"

    # For states with more than 50 universities, we do start=1,51,101,... until total_count
    for start in range(1, total_count + 1, 50):
        params = {
            "dom": domain,
            "key": "",
            "start": str(start)
        }
        # Example: https://univ.cc/search?dom=edu_al&key=&start=1
        response = requests.get(base_search_url, params=params)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")
        ol = soup.find("ol")
        if not ol:
            # No <ol> found, possibly no results
            print(f"No university listing found for {state_name} at start={start}")
            continue

        li_tags = ol.find_all("li")

        for li in li_tags:
            a_tag = li.find("a")
            if not a_tag:
                continue

            university_name = a_tag.get_text(strip=True)
            university_href = a_tag.get("href", "").strip()

            # Keep the URL as is, but make it clickable in Excel.
            # =HYPERLINK("actual_url", "actual_url")
            hyperlink_formula = f'=HYPERLINK("{university_href}", "{university_href}")'

            # Write row: University, State, URL
            writer.writerow([university_name, state_name, hyperlink_formula])

        # Optional delay to avoid sending requests too fast
        # time.sleep(1)

def main():
    # ---- 1) Parse the HTML snippet to get all states  ----
    # In practice, you might load this from a file or from another web request,
    # but here we store it as a string for demonstration:
    html_snippet = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
    </head>
    <body>
        <main>
            <form class="form" method="GET" action="/search">
                <div class="form-row">
                    <label for="dom">Select a State:</label>
                    <select name="dom" size="10">
                        <option value="world" selected="selected">* Search in all U.S. States</option>
                        <option value="edu_al" >Alabama (39)</option>
                        <option value="edu_ak" >Alaska (6)</option>
                        <option value="edu_az" >Arizona (21)</option>
                        <option value="edu_ar" >Arkansas (23)</option>
                        <option value="edu_ca" >California (177)</option>
                        <option value="edu_co" >Colorado (29)</option>
                        <option value="edu_ct" >Connecticut (29)</option>
                        <option value="edu_de" >Delaware (8)</option>
                        <option value="edu_dc" >District of Columbia (15)</option>
                        <option value="edu_fl" >Florida (68)</option>
                        <option value="edu_ga" >Georgia (58)</option>
                        <option value="edu_hi" >Hawaii (9)</option>
                        <option value="edu_id" >Idaho (8)</option>
                        <option value="edu_il" >Illinois (102)</option>
                        <option value="edu_in" >Indiana (51)</option>
                        <option value="edu_ia" >Iowa (34)</option>
                        <option value="edu_ks" >Kansas (25)</option>
                        <option value="edu_ky" >Kentucky (27)</option>
                        <option value="edu_la" >Louisiana (27)</option>
                        <option value="edu_me" >Maine (21)</option>
                        <option value="edu_md" >Maryland (34)</option>
                        <option value="edu_ma" >Massachusetts (78)</option>
                        <option value="edu_mi" >Michigan (61)</option>
                        <option value="edu_mn" >Minnesota (46)</option>
                        <option value="edu_ms" >Mississippi (20)</option>
                        <option value="edu_mo" >Missouri (62)</option>
                        <option value="edu_mt" >Montana (10)</option>
                        <option value="edu_ne" >Nebraska (23)</option>
                        <option value="edu_nv" >Nevada (4)</option>
                        <option value="edu_nh" >New Hampshire (17)</option>
                        <option value="edu_nj" >New Jersey (32)</option>
                        <option value="edu_nm" >New Mexico (11)</option>
                        <option value="edu_ny" >New York (159)</option>
                        <option value="edu_nc" >North Carolina (57)</option>
                        <option value="edu_nd" >North Dakota (16)</option>
                        <option value="edu_oh" >Ohio (90)</option>
                        <option value="edu_ok" >Oklahoma (28)</option>
                        <option value="edu_or" >Oregon (26)</option>
                        <option value="edu_pa" >Pennsylvania (117)</option>
                        <option value="edu_ri" >Rhode Island (10)</option>
                        <option value="edu_sc" >South Carolina (40)</option>
                        <option value="edu_sd" >South Dakota (15)</option>
                        <option value="edu_tn" >Tennessee (49)</option>
                        <option value="edu_tx" >Texas (97)</option>
                        <option value="edu_ut" >Utah (11)</option>
                        <option value="edu_vt" >Vermont (17)</option>
                        <option value="edu_va" >Virginia (53)</option>
                        <option value="edu_wa" >Washington (32)</option>
                        <option value="edu_wv" >West Virginia (23)</option>
                        <option value="edu_wi" >Wisconsin (39)</option>
                        <option value="edu_wy" >Wyoming (4)</option>
                    </select>
                </div>
            </form>
        </main>
    </body>
    </html>
    """

    states_data = parse_states(html_snippet)  # list of (domain, state_name, count)

    # ---- 2) Open CSV and write header ----
    output_csv = "us_universities.csv"
    with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(["University", "State", "URL"])

        # ---- 3) For each state, scrape universities ----
        for domain, state_name, count in states_data:
            print(f"Scraping {state_name} (total {count} universities)...")
            scrape_universities_for_state(domain, state_name, count, writer)
            # time.sleep(1)  # Optional delay if needed

    print(f"Done! Results saved to '{output_csv}'")

if __name__ == "__main__":
    main()


Scraping Alabama (total 39 universities)...
Scraping Alaska (total 6 universities)...
Scraping Arizona (total 21 universities)...
Scraping Arkansas (total 23 universities)...
Scraping California (total 177 universities)...
Scraping Colorado (total 29 universities)...
Scraping Connecticut (total 29 universities)...
Scraping Delaware (total 8 universities)...
Scraping District of Columbia (total 15 universities)...
Scraping Florida (total 68 universities)...
Scraping Georgia (total 58 universities)...
Scraping Hawaii (total 9 universities)...
Scraping Idaho (total 8 universities)...
Scraping Illinois (total 102 universities)...
Scraping Indiana (total 51 universities)...
Scraping Iowa (total 34 universities)...
Scraping Kansas (total 25 universities)...
Scraping Kentucky (total 27 universities)...
Scraping Louisiana (total 27 universities)...
Scraping Maine (total 21 universities)...
Scraping Maryland (total 34 universities)...
Scraping Massachusetts (total 78 universities)...
Scraping M

## MERGE LIST

In [ ]:
import pandas as pd

# Read the first CSV
df_universities = pd.read_csv('universities.csv')
#   Columns are: University, Country, URL

# Read the second CSV
df_us_universities = pd.read_csv('us_universities.csv')
#   Columns are: University, State, URL

# Change 'State' column to 'Country' and set it to 'United States of America'
df_us_universities.rename(columns={'State': 'Country'}, inplace=True)
df_us_universities['Country'] = 'United States of America'

# Concatenate the dataframes
df_combined = pd.concat([df_universities, df_us_universities], ignore_index=True)

# Save back to the first CSV (or a new file, if you prefer)
df_combined.to_csv('universities.csv', index=False)


## GET RANKINGS

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_webometrics_rankings():
    # Base URL template; we will append ?page=X
    base_url = "https://www.webometrics.info/en/world?page={}"

    # Open a CSV file to write results
    with open("world_rankings.csv", mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        # Write CSV header
        writer.writerow(["University", "Ranking"])

        # The site currently shows 74 pages plus page=0 (so 75 pages total)
        for page_num in range(75):
            url = base_url.format(page_num)
            print(f"Scraping page: {url}")
            response = requests.get(url)

            if response.status_code != 200:
                print(f"Warning: Could not retrieve page {page_num}.")
                continue

            soup = BeautifulSoup(response.text, "lxml")
            # The table with the rankings has class "sticky-enabled"
            table = soup.find("table", {"class": "sticky-enabled"})
            if not table:
                # In case no table is found
                print(f"No table found on page {page_num}")
                continue

            # The rows of interest are inside the table body
            tbody = table.find("tbody")
            if not tbody:
                continue

            rows = tbody.find_all("tr")

            for row in rows:
                cols = row.find_all("td")
                # The first column is ranking, second column is the university name
                # Ensure there are enough columns:
                if len(cols) >= 2:
                    ranking = cols[0].get_text(strip=True)
                    university = cols[1].get_text(strip=True)

                    # Write to CSV
                    writer.writerow([university, ranking])

if __name__ == "__main__":
    scrape_webometrics_rankings()


Scraping page: https://www.webometrics.info/en/world?page=0
Scraping page: https://www.webometrics.info/en/world?page=1
Scraping page: https://www.webometrics.info/en/world?page=2
Scraping page: https://www.webometrics.info/en/world?page=3
Scraping page: https://www.webometrics.info/en/world?page=4
Scraping page: https://www.webometrics.info/en/world?page=5
Scraping page: https://www.webometrics.info/en/world?page=6
Scraping page: https://www.webometrics.info/en/world?page=7
Scraping page: https://www.webometrics.info/en/world?page=8
Scraping page: https://www.webometrics.info/en/world?page=9
Scraping page: https://www.webometrics.info/en/world?page=10
Scraping page: https://www.webometrics.info/en/world?page=11
Scraping page: https://www.webometrics.info/en/world?page=12
Scraping page: https://www.webometrics.info/en/world?page=13
Scraping page: https://www.webometrics.info/en/world?page=14
Scraping page: https://www.webometrics.info/en/world?page=15
Scraping page: https://www.webomet

## WITH RANKS

In [ ]:
# import pandas as pd
# import re

# def clean_uni_name(name):
#     """
#     Removes all non-ASCII characters and extra spaces.
#     Convert to lowercase to help ensure matching.
#     """
#     if not isinstance(name, str):
#         return ""
#     # Keep only ASCII letters, digits, and basic punctuation/spaces
#     # Then strip extra whitespace
#     cleaned = re.sub(r'[^\x00-\x7F]+','', name).lower().strip()
#     return cleaned

# # 1. Read the world rankings CSV
# df_rankings = pd.read_csv("world_rankings.csv")
# # Suppose it has columns: University, Ranking

# # 2. Clean the 'University' column in the rankings data
# df_rankings["clean_name"] = df_rankings["University"].apply(clean_uni_name)

# # 3. Read the universities CSV
# df_unis = pd.read_csv("universities.csv")
# # Suppose it has columns: University, Country, URL

# # 4. Clean the 'University' column in the universities data
# df_unis["clean_name"] = df_unis["University"].apply(clean_uni_name)

# # 5. Merge the dataframes on cleaned names, using a left join
# #    So that all rows from df_unis remain, and ranking merges if matched
# df_merged = pd.merge(
#     df_unis,
#     df_rankings[["clean_name", "Ranking"]],
#     on="clean_name",
#     how="left"
# )

# # 6. The merged DataFrame now has a 'Ranking' column
# #    If no match was found, the 'Ranking' will be NaN
# #    (Alternatively, you can fill with an empty string, e.g. fillna(''))
# df_merged["Ranking"] = df_merged["Ranking"].fillna('')

# # 7. Drop the 'clean_name' column if you no longer need it
# df_merged.drop(columns=["clean_name"], inplace=True)

# # 8. Save it back to universities.csv (or a new CSV to avoid overwriting)
# df_merged.to_csv("universities.csv", index=False)


In [ ]:
import re
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

def clean_uni_name(name):
    """
    Removes non-ASCII characters, lowers case, strips whitespace
    """
    if not isinstance(name, str):
        return ""
    return re.sub(r'[^\x00-\x7F]+', '', name).lower().strip()

# ------------------------------------------------------------------------
# 1) Load & clean df_rankings
# ------------------------------------------------------------------------
df_rankings = pd.read_csv("world_rankings.csv", encoding='utf-8')
# Suppose columns: [University, Ranking]
df_rankings["clean_name"] = df_rankings["University"].apply(clean_uni_name)

# Reset index so .iloc[...] works with 0..N-1
df_rankings.reset_index(drop=True, inplace=True)

# ------------------------------------------------------------------------
# 2) Load & clean df_unis
# ------------------------------------------------------------------------
df_unis = pd.read_csv("universities.csv", encoding='utf-8')
# Suppose columns: [University, Country, URL] (and possibly 'Ranking' from before).

# If the original 'universities.csv' also has a 'Ranking' column,
# rename it to avoid collisions:
if "Ranking" in df_unis.columns:
    df_unis.rename(columns={"Ranking": "OriginalRanking"}, inplace=True)

df_unis["clean_name"] = df_unis["University"].apply(clean_uni_name)

# ------------------------------------------------------------------------
# 3) Merge on "clean_name" for direct matches
# ------------------------------------------------------------------------
df_merged = pd.merge(
    df_unis,
    df_rankings[["clean_name", "University", "Ranking"]],
    on="clean_name",
    how="left",
    suffixes=("", "_ranks")  # e.g., 'Ranking_ranks' for the right table
)

# ------------------------------------------------------------------------
# 4) Create a new "Matched Uni" column from direct merges
# ------------------------------------------------------------------------
df_merged["Matched Uni"] = df_merged["University_ranks"]

# ------------------------------------------------------------------------
# 5) Finalize the single "Ranking" column
# ------------------------------------------------------------------------
# - If "Ranking_ranks" exists (the column from df_rankings),
#   then adopt it as our main "Ranking" column.
#   Otherwise, create "Ranking" if it doesn't exist yet.
if "Ranking_ranks" in df_merged.columns:
    # Where 'Ranking_ranks' is not NaN, use it
    df_merged["Ranking"] = df_merged["Ranking_ranks"]
    # Now drop the "Ranking_ranks" column
    df_merged.drop(columns=["Ranking_ranks"], inplace=True)

# If "Ranking" did not exist at all (e.g. if we never had a collisions), ensure it does:
if "Ranking" not in df_merged.columns:
    df_merged["Ranking"] = np.nan

# Convert "Ranking" & "Matched Uni" to string so we can assign ""
df_merged["Ranking"] = df_merged["Ranking"].astype(str)
df_merged["Matched Uni"] = df_merged["Matched Uni"].astype(str)

# ------------------------------------------------------------------------
# 6) Embeddings fallback for rows where "Ranking" is missing
# ------------------------------------------------------------------------
missing_mask = (
    df_merged["Ranking"].eq("nan") |
    df_merged["Ranking"].eq("None") |
    df_merged["Ranking"].eq("")
)
if missing_mask.any():
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    # Subset those rows
    df_missing = df_merged.loc[missing_mask].copy()

    # Encode their clean_names
    missing_emb = model.encode(df_missing["clean_name"].tolist(), convert_to_tensor=True)

    # Encode all ranking unis
    rank_emb = model.encode(df_rankings["clean_name"].tolist(), convert_to_tensor=True)

    # Fallback search
    for i, row_idx in enumerate(df_missing.index):
        this_emb = missing_emb[i]

        # Cosine similarities
        cos_scores = util.cos_sim(this_emb, rank_emb)[0]  # shape: [N]
        best_idx = cos_scores.argmax().item()
        best_score = float(cos_scores[best_idx].item())

        # Lower threshold to 0.70
        if best_score >= 0.70:
            matched_name = df_rankings.iloc[best_idx]["University"]
            matched_rank = df_rankings.iloc[best_idx]["Ranking"]
            df_merged.at[row_idx, "Matched Uni"] = matched_name
            df_merged.at[row_idx, "Ranking"] = str(matched_rank)
        else:
            df_merged.at[row_idx, "Matched Uni"] = ""
            df_merged.at[row_idx, "Ranking"] = ""

# ------------------------------------------------------------------------
# 7) Tidy up leftover columns & save
# ------------------------------------------------------------------------
# "University_ranks" is from the merge. Remove it if still present.
if "University_ranks" in df_merged.columns:
    df_merged.drop(columns=["University_ranks"], inplace=True)

# Also remove the "clean_name" helper
if "clean_name" in df_merged.columns:
    df_merged.drop(columns=["clean_name"], inplace=True)

# Save final
df_merged.to_csv("universities_4.csv", index=False)

print("Done! 'universities_4.csv' now has a single 'Ranking' column and a 'Matched Uni' column, with no leftover '_ranks' columns.")


Done! 'universities_4.csv' now has a single 'Ranking' column and a 'Matched Uni' column, with no leftover '_ranks' columns.


In [ ]:
#df_merged.drop(columns=["clean_name", "University_ranks"], inplace=True, errors="ignore")
df_merged.to_csv("universities_2.csv", index=False)

In [ ]:
df_merged.head(10)

,University,Country,URL,Ranking,Ranking_ranks,Matched Uni
0,1 December University of Alba Iulia,Romania,"=HYPERLINK(""http://www.uab.ro/"", ""http://www.u...",,NaN,
1,2nd Military Medical University,China,"=HYPERLINK(""http://www.smmu.edu.cn/"", ""http://...",1411,NaN,The Second Military Medical University / 第二军医大学
2,3rd Military Medical University,China,"=HYPERLINK(""http://www.tmmu.edu.cn/"", ""http://...",1915,NaN,Third Military Medical University / 第三军医大学
3,4th Military Medical University,China,"=HYPERLINK(""https://www.fmmu.edu.cn/"", ""https:...",1330,NaN,The Fourth Military Medical University / 第四军医大学
4,AAB College,Kosovo,"=HYPERLINK(""https://aab-edu.net/"", ""https://aa...",,NaN,
5,Aalborg Business College,Denmark,"=HYPERLINK(""http://www.ah.dk/"", ""http://www.ah...",,NaN,
6,Aalborg University,Denmark,"=HYPERLINK(""http://www.aau.dk/"", ""http://www.a...",,NaN,
7,Aalto University,Finland,"=HYPERLINK(""http://www.aalto.fi/"", ""http://www...",,NaN,
8,Aarhus School of Architecture,Denmark,"=HYPERLINK(""http://www.a-aarhus.dk/"", ""http://...",,NaN,
9,Aarhus School of Business,Denmark,"=HYPERLINK(""http://bss.au.dk/"", ""http://bss.au...",,NaN,
